In [1]:
include("optim_lib.jl")
include("lib_func.jl")

# Constants and Parameters
d1, d2, ds = 3, 8, 1  # Degree range and step
const n, a, b = 4, 10, 1 
const C = a / b  # Scaling constant, C is appears in `main_computation`, maybe it should be a parameter.
const delta, alph = .9 , 2 / 10  # Sampling parameters
f = shubert_4d # Objective function

shubert_4d (generic function with 1 method)

In [2]:
coeffs_poly_approx = main_gen(f, n, d1, d2, ds, delta, alph, C, 0.2)

dimension Vector space: 35
sample size: 256
dimension Vector space: 70
sample size: 256
dimension Vector space: 126
sample size: 625
dimension Vector space: 210
sample size: 1296
dimension Vector space: 330
sample size: 1296
dimension Vector space: 495
sample size: 2401


6-element Vector{Vector{Float64}}:
 [0.06249014079223168, -0.5357054864177533, 0.25841609771318813, 0.25363619681339755, -0.5357054864177538, 9.184820662582268, -4.4306164002380015, 0.25841609771318813, -4.430616400238001, 0.25363619681339705  …  2.358198851602585e-16, 9.184820662582272, -1.6043023390711187e-15, 1.6299302579437611e-16, -4.430616400238006, 0.2584160977131883, 2.3322545246911183e-16, 2.282898679726866e-16, -4.430616400238002, 0.2536361968133976]
 [-3.3457849565942133, -0.9852754849692609, -0.5722809424725449, 1.3389941845524993, -7.054336497542864e15, -0.00047785360292995627, 9.184820662582268, -4.4306164002380015, -4.34866366004288, 1.2473878080973526  …  -0.0, -4.430616400238001, 8.881784197001252e-16, 4.85722573273506e-17, 2.137261293084298, 0.004453871042878957, 1.6361926408190708e-16, 4.473256443070617e-16, -4.348663660042881, 1.619572523612494e15]
 [0.08751375096346145, -0.45874699577773664, 0.2170891319185766, -1.0699638560088014, 0.8622744446872557, -2.6654802845

In [3]:
using DynamicPolynomials, MultivariatePolynomials, HomotopyContinuation, ProgressLogging, DataFrames

include("hom_solve.jl") # Include the homotopy solver and main function
vars = @polyvar(x[1:n]) # Define polynomial ring 
h_x, h_y, h_z, h_t, col = main_4d(n, d1, d2, ds, coeffs_poly_approx) # main_2d is in hom_solve.jl
df = DataFrame(x = C*h_x, y = C*h_y, z = C*h_z, t = C*h_t, col=col)
df[!, :result] = [f([df.x[i], df.y[i], df.z[i], df.t[i]]) for i in 1:nrow(df)];
println(df)


Computing mixed cells... 2    Time: 0:00:00
Computing mixed cells... 3    Time: 0:00:01
  mixed_volume:  16


Tracking 16 paths...  12%|███▊                          |  ETA: 0:00:48



Tracking 16 paths... 100%|██████████████████████████████| Time: 0:00:07
  # paths tracked:                  16
  # non-singular solutions (real):  16 (4)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         16 (4)
Degree: 3
Number of solutions: 4
Tracking 81 paths...   2%|▊                             |  ETA: 0:02:13



Tracking 81 paths... 100%|██████████████████████████████| Time: 0:00:03
  # paths tracked:                  81
  # non-singular solutions (real):  81 (81)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         81 (81)
Degree: 4
Number of solutions: 81
Tracking 256 paths...   1%|▎                            |  ETA: 0:11:10



Tracking 256 paths...  42%|████████████▏                |  ETA: 0:00:08



Tracking 256 paths...  80%|███████████████████████▏     |  ETA: 0:00:01



Tracking 256 paths... 100%|█████████████████████████████| Time: 0:00:05
  # pa

Now we use Optim.jl package to initialize step 3 and converge to the local minimizers.
How do we know we have everything ? Use simple combinations of 

In [5]:
using Optim 

# for i in 1:nrow(df)
#     println("Optimizing for point $i")
#     x0 = [df.x[i], df.y[i], df.z[i], df.t[i]]
#     res = Optim.optimize(f, x0, LBFGS(), Optim.Options(show_trace=true))
#     println(summary(res))
# end

# Assuming df is already defined and contains columns: x, y, z, t

# Create new columns for the minimizer points and the minimum value


for i in 1:nrow(df)
    println("Optimizing for point $i")
    x0 = [df.x[i], df.y[i], df.z[i], df.t[i]]
    res = Optim.optimize(f, x0, LBFGS(), Optim.Options(show_trace=true))
    minimizer = Optim.minimizer(res)
    min_value = Optim.minimum(res)
    println(summary(res))

    # Append the minimizer points and the minimum value to the DataFrame
    push!(df.min_x, minimizer[1])
    push!(df.min_y, minimizer[2])
    push!(df.min_z, minimizer[3])
    push!(df.min_t, minimizer[4])
    push!(df.min_value, min_value)
end

# Display the updated DataFrame
using AceTools
display_dataframe_to_user("Updated DataFrame with Minimizer Points and Minimum Value", df)


ArgumentError: ArgumentError: New columns must have the same length as old columns